# Dataloader experiments

In [3]:
import open3d as o3d
import numpy as np
import matplotlib.pyplot as plt

from pointnet2_ops import pointnet2_utils
from knn_cuda import KNN

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


/home/vinaylanka/miniconda3/envs/obitonet/lib/python3.10/site-packages/torch/utils/cpp_extension.py:1964: UserWarning: TORCH_CUDA_ARCH_LIST is not set, all archs for visible cards are included for compilation. 
If this is not desired, please set os.environ['TORCH_CUDA_ARCH_LIST'].
  warnings.warn(


## 3D PCD: Importing, Downsampling and Saving as numpy arrays

In [17]:
print("Loading Open3D point cloud")
path = "../dataset/Dataset/Barn_is/Barn/Barn02.ply"
pcd = o3d.io.read_point_cloud(path)
print("PCD Details:")
print(pcd)

# Link: https://www.open3d.org/docs/latest/tutorial/Advanced/pointcloud_outlier_removal.html
print("Every 100th points are selected, could use voxel or uniform downsampling")
# down_pcd = pcd.voxel_down_sample(voxel_size=0.02)
down_pcd = pcd.uniform_down_sample(every_k_points=100)

#Link: https://www.open3d.org/html/tutorial/Basic/working_with_numpy.html
np_pcd = np.asarray(down_pcd.points)
print(np_pcd.shape)

#Visualise PCD
o3d.visualization.draw_geometries([down_pcd])


Loading Open3D point cloud
PCD Details:
PointCloud with 3305027 points.
Every 100th points are selected, could use voxel or uniform downsampling
(33051, 3)
